## I. Chương trình RAG

In [1]:
# 1. Cài đặt các gói thư viện cần thiết
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4
!pip install -q accelerate==0.22.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 843.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [9]:
#2. Xây dựng vector database
# (a) Import các thư viện cần thiết
import torch

from transformers import BitsAndBytesConfig # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # type: ignore
from langchain_huggingface import HuggingFaceEmbeddings # type: ignore
from langchain_huggingface.llms import HuggingFacePipeline # type: ignore

from langchain.memory import ConversationBufferMemory # type: ignore
from langchain_community.chat_message_histories import ChatMessageHistory # type: ignore
from langchain_community.document_loaders import PyPDFLoader, TextLoader # type: ignore
from langchain.chains import ConversationalRetrievalChain # type: ignore

from langchain_chroma import Chroma # type: ignore
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langchain import hub # type: ignore

import accelerate # type: ignore
from accelerate import init_empty_weights # type: ignore


In [3]:
# (b) Đọc file pdf: Từ một file pdf
Loader = PyPDFLoader
FILE_PATH = "./YOLOv10_Tutorials.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [4]:
# (c) Khởi tạo bộ tách văn bản (text splitter)
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap =100)

# Tách file pdf
docs = text_splitter.split_documents(documents)

print("Number of sub - documents: ", len(docs))
print(docs[0])

Number of sub - documents:  33
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt 

In [5]:
# (d) Khởi tạo instance vectorization để chuyển đổi văn bản thành vector
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# (e) Khởi tạo vector database
vector_db = Chroma.from_documents(documents =docs, embedding = embedding)

retriever = vector_db.as_retriever()

# Thực hiện truy vấn với một đoạn văn bản bất kì tại đây:
result = retriever.invoke("What is YOLO ?")
print("Number of relevant documents: ", len(result))

Number of relevant documents:  4


In [7]:
# 3. Khởi tạo mô hình ngôn ngữ lớn
# (a) Khai báo một số cài đặt cần thiết cho mô hình
nf4_config = BitsAndBytesConfig (
                                load_in_4bit = True,
                                bnb_4bit_quant_type = "nf4",
                                bnb_4bit_use_double_quant = True,
                                bnb_4bit_compute_dtype = torch.bfloat16
                                )

In [11]:
# (b) Khởi tạo mô hình và tokenizer
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
                                            MODEL_NAME,
                                            quantization_config = nf4_config,
                                            low_cpu_mem_usage=True
                                            )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [14]:
# (c) Tích hợp tokenizer và model thành một pipeline để tiện sử dụng:
model_pipeline = pipeline(
                        "text-generation",
                        model  = model,
                        tokenizer = tokenizer,
                        max_new_tokens = 512,
                        pad_token_id = tokenizer.eos_token_id,
                        device_map ="auto"
                        )

llm = HuggingFacePipeline(pipeline = model_pipeline)

In [16]:
# 4. Chạy chương trình
prompt = hub. pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
                {"context": retriever | format_docs, "question": RunnablePassthrough()}
                | prompt
                | llm
                | StrOutputParser()
            )

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.


## II. Xây dựng giao diện chat

In [ ]:
# 1. Tải các gói thư viện:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.22.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain-openai==0.1.9
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!npm install -g localtunnel
!pip install -q numpy==1.24.4

In [18]:
!pip install -q chainlit==1.1.304

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.3 MB/s eta 0:00:00


In [19]:
# 2. Import các gói thư viện cần thiết
import chainlit as cl # type: ignore
import torch

from chainlit.types import AskFileResponse # type: ignore

from transformers import BitsAndBytesConfig # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # type: ignore
from langchain_huggingface.llms import HuggingFacePipeline # type: ignore

from langchain.memory import ConversationBufferMemory # type: ignore
from langchain_community.chat_message_histories import ChatMessageHistory # type: ignore
from langchain.chains import ConversationalRetrievalChain # type: ignore
from langchain_huggingface import HuggingFaceEmbeddings # type: ignore
from langchain_chroma import Chroma # type: ignore
from langchain_community.document_loaders import PyPDFLoader, TextLoader # type: ignore
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langchain import hub # type: ignore

In [20]:
# 3. Cài đặt lại các hàm và instance ở file trước
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap =100)

embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
# 4. Xây dựng hàm xử lý file input đầu vào
def process_file(file : AskFileResponse):
    if file.type == "text/plain":
        Loader = TextLoader
    elif file.type == "application/pdf":
        Loader = PyPDFLoader

    loader = Loader(file.path)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)
    for i, doc in enumerate(docs):
        doc.metadata["source"] = f"source_{i}"
    return docs

In [22]:
# 5. Xây dựng hàm khởi tạo Chroma database
def get_vector_db(file : AskFileResponse) :
    docs = process_file(file)
    cl.user_session.set("docs", docs)
    vector_db = Chroma.from_documents(documents =docs, embedding = embedding)
    return vector_db

In [25]:
# 6. Khởi tạo mô hình ngôn ngữ lớn:
def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5", max_new_token: int = 512):
    nf4_config = BitsAndBytesConfig(
                                    load_in_4bit =True,
                                    bnb_4bit_quant_type ="nf4",
                                    bnb_4bit_use_double_quant =True ,
                                    bnb_4bit_compute_dtype = torch.bfloat16
                                    )
    model = AutoModelForCausalLM.from_pretrained(
                                                model_name,
                                                quantization_config = nf4_config,
                                                low_cpu_mem_usage = True
                                                )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline (
                                "text-generation",
                                model =model,
                                tokenizer = tokenizer,
                                max_new_tokens = max_new_token,
                                pad_token_id = tokenizer.eos_token_id,
                                device_map ="auto"
                                )

    llm = HuggingFacePipeline (pipeline = model_pipeline)
    return llm

LLM = get_huggingface_llm()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [26]:
# 7. Khởi tạo welcome message
welcome_message = """ Welcome to the PDF QA! To get started :
                        1. Upload a PDF or text file
                        2. Ask a question about the file
                        """

In [27]:
# 8. Khởi tạo hàm on_chat_start:
@cl.on_chat_start
async def on_chat_start () :
    files = None
    while files is None :
        files = await cl.AskFileMessage(
                                        content = welcome_message ,
                                        accept =["text/plain", "application/pdf"] ,
                                        max_size_mb =20,
                                        timeout =180,
                                        ).send()
    file = files [0]

    msg = cl.Messag(content =f"Processing '{file.name}'...", disable_feedback = True)
    await msg.send()

    vector_db = await cl.make_async(get_vector_db)(file)

    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
                                        memory_key ="chat_histor ",
                                        output_key ="answer",
                                        chat_memory = message_history,
                                        return_messages =True,
                                    )
    retriever = vector_db.as_retriever(search_type ="mmr", search_kwargs ={'k': 3})

    chain = ConversationalRetrievalChain.from_llm(
                                                    llm =LLM,
                                                    chain_type ="stuff ",
                                                    retriever = retriever,
                                                    memory =memory,
                                                    return_source_documents = True
                                                    )

    msg.content = f"'{file.name} 'processed.You can now ask questions!"
    await msg.update()

    cl.user_session.set("chain", chain)

In [30]:
# 9. Khởi tạo hàm on_message:
@cl.on_message
async def on_message(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler()
    res = await chain.ainvoke(message.content, callbacks=[cb])
    answer = res["answer"]
    source_documents = res["source_documents"]
    text_elements = []

    if source_documents:
        for source_idx, source_doc in enumerate(source_documents):
            source_name = f"source_{source_idx}"
            text_elements.append(
                                cl.Text(content = source_doc.page_content,
                                        name = source_name)
                                )
        source_names = [text_el.name for text_el in text_elements]

        if source_names :
            answer += f"\nSources: {','.join(source_names)}"
        else :
            answer += "\nNo sources found"

    await cl.Message(content = answer, elements = text_elements).send()

In [33]:
# 10. Chạy chainlit app
!chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

In [35]:
!apt-get update
!apt-get install -y nodejs npm
!npm install -g localtunnel

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,116 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,566 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com

In [36]:
# 11. Expose localhost thành public host bằng localtunnel:
import urllib

print ("Password/Enpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!lt --port 8000 -- subdomain aivn - simple - rag

Password/Enpoint IP for localtunnel is: 34.83.164.151
your url is: https://curly-beers-sort.loca.lt
^C
